In [94]:
# imports
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
import rtree
import difflib
from io import StringIO

# show graphs & charts inline
%matplotlib inline 
plt.rcParams["figure.figsize"] = (15, 7) # define plot size
saved_style_state = matplotlib.rcParams.copy() # give us a style state to go back to

In [120]:
df_penaltyData = pd.read_csv("data/penalty_data_set_0.csv")
print("done")

done


In [121]:
cols_to_drop = ['OFFENCE_MONTH', 'OFFENCE_CODE', 'OFFENCE_DESC',
        'SECTION_CLAUSE', 'LOCATION_CODE','SCHOOL_ZONE_IND',
       'SPEED_BAND', 'SPEED_IND', 'POINT_TO_POINT_IND', 'RED_LIGHT_CAMERA_IND',
       'SPEED_CAMERA_IND', 'SEATBELT_IND', 'MOBILE_PHONE_IND', 'PARKING_IND',
       'CINS_IND', 'FOOD_IND', 'BICYCLE_TOY_ETC_IND']

df_penaltyData = df_penaltyData.drop(cols_to_drop, axis=1)
df_penaltyData.head(2)

,OFFENCE_FINYEAR,LEGISLATION,FACE_VALUE,CAMERA_IND,CAMERA_TYPE,LOCATION_DETAILS,TOTAL_NUMBER,TOTAL_VALUE
0,2011-2012,ROAD RULES 2008,353,Y,Red Light / Speed Camera,VICTORIA ROAD ROZELLE EASTBOUND,46,16238
1,2011-2012,ROAD RULES 2008,265,NaN,NaN,NaN,78,20670


In [122]:
df_penaltyData.shape

(246305, 8)

In [123]:
is_cam = df_penaltyData["CAMERA_IND"]=="Y"

df_penaltyData = df_penaltyData[is_cam]
df_penaltyData.shape

(157760, 8)

In [124]:
df_cams = pd.read_csv("data/Fixed Speed Cameras.csv")
print("done")

done


In [125]:
df_cams.head(2)

,SZ?,SUBURB/TOWN,ROAD/S,Cameras,Lat(1),Long(1),Lat(2),Long(2)
0,Y,Ashfield,"Hume Highway, between Murrell Street and Queen...",1,-33.890177,151.127985,NaN,NaN
1,N,Auburn,"Parramatta Road, between Harbord Street and Du...",1,-33.835963,151.023621,NaN,NaN


In [126]:
df_cams["LOCATION_DETAILS"] = df_cams["SUBURB/TOWN"] + " " + df_cams["ROAD/S"]

In [127]:
df_cams["LOCATION_DETAILS"] = df_cams["LOCATION_DETAILS"].str.upper().str.split(",").str[0]
df_cams = df_cams.drop(["SZ?","SUBURB/TOWN", "ROAD/S", "Lat(2)", "Long(2)"], axis=1)
df_cams.head(2)

,Cameras,Lat(1),Long(1),LOCATION_DETAILS
0,1,-33.890177,151.127985,ASHFIELD HUME HIGHWAY
1,1,-33.835963,151.023621,AUBURN PARRAMATTA ROAD


In [128]:
gdf_cams = gpd.GeoDataFrame(df_cams, geometry=gpd.points_from_xy(df_cams["Long(1)"], df_cams["Lat(1)"]))
gdf_cams.head(2)

,Cameras,Lat(1),Long(1),LOCATION_DETAILS,geometry
0,1,-33.890177,151.127985,ASHFIELD HUME HIGHWAY,POINT (151.12799 -33.89018)
1,1,-33.835963,151.023621,AUBURN PARRAMATTA ROAD,POINT (151.02362 -33.83596)


In [129]:
df_penaltyData.head(2)

,OFFENCE_FINYEAR,LEGISLATION,FACE_VALUE,CAMERA_IND,CAMERA_TYPE,LOCATION_DETAILS,TOTAL_NUMBER,TOTAL_VALUE
0,2011-2012,ROAD RULES 2008,353,Y,Red Light / Speed Camera,VICTORIA ROAD ROZELLE EASTBOUND,46,16238
5,2011-2012,ROAD RULES 2008,93,Y,Fixed Digital Speed Camera,HUME HIGHWAY ASHFIELD (Z) WESTBOUND,9,837


In [135]:
cams = list(gdf_cams["LOCATION_DETAILS"])

